In [1]:
import pandas as pd
import sqlite3

## • create a connection to the database using the library sqlite3

In [2]:
con = sqlite3.connect('../data/checking-logs.sqlite.sqlite copy')

## • get the schema of the table test

In [3]:
table = 'test'
query = f'PRAGMA table_info({table});'
pd.io.sql.read_sql(query, con)

,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,uid,TEXT,0,None,0
2,2,labname,TEXT,0,None,0
3,3,first_commit_ts,TIMESTAMP,0,None,0
4,4,first_view_ts,TIMESTAMP,0,None,0


## • get only the first 10 rows of the table checker to check what the table looks like

In [4]:
count = 10
firstRows = f'SELECT * FROM {table} LIMIT {count};'
pd.io.sql.read_sql(firstRows, con)

,index,uid,labname,first_commit_ts,first_view_ts
0,3,user_17,project1,2020-04-18 07:56:45.408648,2020-04-18 10:56:55.833899
1,4,user_30,laba04,2020-04-18 13:36:53.971502,2020-04-17 22:46:26.785035
2,7,user_30,laba04s,2020-04-18 14:51:37.498399,2020-04-17 22:46:26.785035
3,8,user_14,laba04,2020-04-18 15:14:00.312338,2020-04-18 10:53:52.623447
4,11,user_14,laba04s,2020-04-18 22:30:30.247628,2020-04-18 10:53:52.623447
5,18,user_19,laba04,2020-04-20 19:05:01.297780,2020-04-21 20:30:38.034966
6,19,user_25,laba04,2020-04-20 19:16:50.673054,2020-05-09 23:54:54.260791
7,20,user_21,laba04,2020-04-21 17:48:00.487806,2020-04-22 22:40:36.824081
8,21,user_30,project1,2020-04-22 12:36:24.053518,2020-04-17 22:46:26.785035
9,23,user_21,laba04s,2020-04-22 20:09:21.857747,2020-04-22 22:40:36.824081


[Разница между двумя датами в Sqlite3](https://coderoad.ru/55543509/%D0%A0%D0%B0%D0%B7%D0%BD%D0%B8%D1%86%D0%B0-%D0%BC%D0%B5%D0%B6%D0%B4%D1%83-%D0%B4%D0%B2%D1%83%D0%BC%D1%8F-%D0%B4%D0%B0%D1%82%D0%B0%D0%BC%D0%B8-%D0%B2-Sqlite3)

[Разница между двумя датами в SQLite](https://overcoder.net/q/1305/%D1%80%D0%B0%D0%B7%D0%BD%D0%B8%D1%86%D0%B0-%D0%BC%D0%B5%D0%B6%D0%B4%D1%83-%D0%B4%D0%B2%D1%83%D0%BC%D1%8F-%D0%B4%D0%B0%D1%82%D0%B0%D0%BC%D0%B8-%D0%B2-sqlite)

[SQLite: julianday Function](https://www.techonthenet.com/sqlite/functions/julianday.php)

In [9]:
aggregated_query = """
SELECT  uid,
        MIN(julianday(test.first_commit_ts - deadlines.deadlines)) AS delta
FROM test
"""

pd.io.sql.read_sql(aggregated_query, con)


DatabaseError: Execution failed on sql '
SELECT  uid,
        MIN(julianday(test.first_commit_ts - deadlines.deadlines)) AS delta
FROM test
': no such column: deadlines.deadlines

In [ ]:
con.close()